<a href="https://colab.research.google.com/github/sczopek/spaceInvadersAtariRl/blob/main/copy_of_atari_games.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Baselines3 - Train on Atari Games

Github Repo: [https://github.com/DLR-RM/stable-baselines3](https://github.com/DLR-RM/stable-baselines3)


[RL Baselines3 Zoo](https://github.com/DLR-RM/rl-baselines3-zoo) is a training framework for Reinforcement Learning (RL), using Stable Baselines3.

It provides scripts for training, evaluating agents, tuning hyperparameters, plotting results and recording videos.

Documentation is available online: [https://stable-baselines3.readthedocs.io/](https://stable-baselines3.readthedocs.io/)

## Install Dependencies and Stable Baselines Using Pip


```
pip install stable-baselines3[extra]
```

In [ ]:
!pip install "stable-baselines3[extra]<=2.3.1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 22.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl.metadata (558 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.9/182.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.2 MB/s eta 0:00:00
Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446661 sha256=872b5f2a44cadb3f9351f8b84e04a9a0f73e384103b4b0eaa849c68a082ba5a1
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license


## Import policy, RL agent, ...

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack

In [ ]:
from google.colab import files

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Training on Atari

We will use atari wrapper (it will downsample the image and convert it to gray scale).

About Atari preprocessing: [Frame Skipping and Pre-Processing for Deep Q-Networks on Atari 2600 Games](https://danieltakeshi.github.io/2016/11/25/frame-skipping-and-preprocessing-for-deep-q-networks-on-atari-2600-games/)

![Pong](https://cdn-images-1.medium.com/max/800/1*UHYJE7lF8IDZS_U5SsAFUQ.gif)

In [ ]:
# There already exists an environment generator that will make and wrap atari environments correctly.
env = make_atari_env("SpaceInvaders-v4", n_envs=8, seed=0)
# Stack 4 frames
vec_env = VecFrameStack(env, n_stack=4)

In [ ]:
import os
models_dir = "models/PPO"
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from stable_baselines3.common.callbacks import CheckpointCallback

# use CheckpointCallback to save model during training
# copy of model at training checkpoint is written to google drive

# Save a copy of the model when training reaches a checkpoint
#  checkpoint = every 2_000_000 frames of training
# 8 env running simultaneously, so set save_freq=250000
# save_freq*8 = 2_000_000
checkpoint_callback = CheckpointCallback(save_freq=250000, save_path="/content/gdrive/MyDrive/models/PPO",
                                         name_prefix="ppo_spaceInvadersFrameSkipWinningParams_136000000_steps")

In [ ]:
# if not loading model, make new model here to start training from scratch
# hyper params for space invaders model as inputs
# model = PPO("CnnPolicy", vec_env, verbose=1, tensorboard_log="/content/gdrive/MyDrive/models/ppo_spaceinvaders_tensorboard/",
#             batch_size=256,
#             clip_range=0.001,
#             ent_coef=0.01,
#             learning_rate=0.0001,
#             n_epochs=4,
#             n_steps=128,
#             vf_coef=0.5
#             )

Using cuda device
Wrapping the env in a VecTransposeImage.


In [ ]:
model = PPO.load("/content/gdrive/MyDrive/models/PPO/ppo_spaceInvadersFrameSkipWinningParams_136000000_steps",
                 verbose=1,
                 tensorboard_log="/content/gdrive/MyDrive/models/ppo_spaceinvaders_tensorboard/",
                 force_reset=False)
model.set_env(vec_env)

Wrapping the env in a VecTransposeImage.


In [ ]:
# can train from 20 Millions frames before booted from google colab server
# for excessive connection time
model.learn(total_timesteps=20000000, callback=[checkpoint_callback], reset_num_timesteps=False)

Streaming output truncated to the last 5000 lines.
|    value_loss           | 0.87         |
------------------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 9.79e+03      |
|    ep_rew_mean          | 2.39e+03      |
| time/                   |               |
|    fps                  | 555           |
|    iterations           | 29060         |
|    time_elapsed         | 53593         |
|    total_timesteps      | 63712528      |
| train/                  |               |
|    approx_kl            | 0.00065765623 |
|    clip_fraction        | 0.861         |
|    clip_range           | 0.001         |
|    entropy_loss         | -1.34         |
|    explained_variance   | 0.943         |
|    learning_rate        | 0.0001        |
|    loss                 | 0.0635        |
|    n_updates            | 266444        |
|    policy_gradient_loss | 0.0013        |
|    value_loss           |

Save and download trained model